In [1]:
import pandas as pd
import numpy as np

#pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 50)

df = pd.read_csv('posts.tsv', sep='\t', header=0)
print(df.shape)
df.head()

(26437, 9)


,post_id,response_id,timestamp,url,total_comment,comment_count,actor_id,actor_url,message_text
0,0,SEO_PPC_Tech_support/Response/response_82.json,1610004359,https://www.facebook.com/groups/44233687260999...,0,0,100032384902900,https://www.facebook.com/people/Akhil-Sinha-Re...,*namaste*🙏🤝 🇺🇸🇨🇦🇬🇧🇰🇾🆗 *blocking pickup service...
1,1,SEO_PPC_Tech_support/Response/response_82.json,1609999274,https://www.facebook.com/groups/44233687260999...,0,0,100004129564748,https://www.facebook.com/people/Shubham-Mishra...,cashapp available in gud flow pure ppc calls ...
2,2,SEO_PPC_Tech_support/Response/response_82.json,1609984311,https://www.facebook.com/groups/44233687260999...,0,0,100053502061434,https://www.facebook.com/jason.watts.9638718,email blasting data ivr data press 1 data and ...
3,3,SEO_PPC_Tech_support/Response/response_451.json,1604925322,https://www.facebook.com/groups/44233687260999...,0,0,100031520135794,https://www.facebook.com/people/Piryank-Sharma...,☠️ zelle and gpay 👉instant payment👈 ☠️ cash ap...
4,4,SEO_PPC_Tech_support/Response/response_451.json,1604924959,https://www.facebook.com/groups/44233687260999...,0,0,100055968734764,https://www.facebook.com/people/Sahil-Singh/10...,"*hello centers*, we are providing services ac..."


In [2]:
import re
import phonenumbers

def has_numbers(input_string):
    numbers = re.findall('(\d)', input_string)
    if len(numbers) == 0:
        return False
    
    if len(numbers) == 1: 
        if phonenumbers.PhoneNumberMatcher(input_string, 'US'):
            return False
    
    return True
    

df['has_digit'] = df['message_text'].apply(has_numbers) 
df[df['has_digit'] == True].shape

(12640, 10)

In [13]:
def empty_post(msg):
    if msg == 'na':
         return True
    return False

df['empty_post'] = df['message_text'].apply(empty_post)

df = df[df['empty_post'] == False]
print(df.shape)
print(pd.pivot_table(df, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(df, index=['actor_id'], aggfunc=pd.Series.count).shape)
print(df['total_comment'].sum(axis=0, skipna=True))

(22043, 18)
(7596, 17)
(2229, 17)
14415


In [4]:
def money_laundering(msg):
    
    available = 'available' in msg
    payment = 'payment' in msg or 'payout' in msg
    blocking = 'blocking' in msg
    
    giftcard = 'gift card' in msg or 'giftcard' in msg or 'gifts card' in msg 
    if giftcard and (blocking or available or payment):
        return 'giftcard'
    
    
    gateway = 'gatewy' in msg or 'gateway' in msg or 'gatway' in msg
    payment_form = gateway or payment or 'loader' in msg or 'merchant' in msg or 'link' in msg
    two_d = '2d' in msg or '2-d' in msg or '2 d' in msg or '3d' in msg or '3-d' in msg or '3 d' in msg or 'e check' in msg
    if two_d and payment_form:
        return '2d or 3d gateway'
    
    
    bank = 'bank account' in msg
    if bank:
        return 'bank account'
    
    
    seller = 'sell' in msg or 'buy' in msg or 'trader' in msg or 'flash' in msg
    bitcoin = 'btc' in msg
    if bitcoin and seller:
        return 'bitcoin'
    
    
    loader = 'loader' in msg
    card = 'mastercard' in msg or 'americanexpress' in msg or 'amex' in msg
    if card and loader and available:
        return 'card'
    
    
    different_payments = 'chime' in msg or 'paypal' in msg or 'zelle' in msg or 'g-pay' in msg or 'apple' in msg
    if different_payments and payment:
        return 'other payment forms'
    
    if blocking and (payment or available):
        return 'blocking available'
    
    return False


df['money_laundering'] = df['message_text'].apply(money_laundering)
temp = df[df['money_laundering'] != False]

print(temp.shape)

print(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)
print(temp['total_comment'].sum(axis=0, skipna=True))

(6677, 12)
(1435, 11)
(541, 11)
3097


In [5]:
def website_developer(msg):
    
    available = 'avail' in msg or 'active' in msg or 'running' in msg or 'book' in msg or 'live' in msg or ' cc ' in msg or 'order more' in msg or 'direct center' in msg
    call = 'call' in msg or 'cll' in msg
    
    call_or_available = call and available
    
    google_ads = ('adword' in msg or 'ppc' in msg) and ('campaign' in msg or 'account' in msg or call or available)
    if google_ads:
        return 'google ads'
    
    
    if 'amazon' in msg and call_or_available:
        return 'amazon'
    

    if 'ios' in msg and call_or_available:
        return 'ios'
    
    
    if 'paypal' in msg and call_or_available:
        return 'paypal'
    
   
    if 'chime' in msg and call_or_available:
        return 'chime'
    
   
    if ('cashapp' in msg or 'cash app' in msg) and call_or_available:
        return 'cashapp'
    
    
    if ('quickbook' in msg) and call_or_available:
        return 'quickbook'
    
    
    if 'printer' in msg and call_or_available:
        return 'printer'
    
   
    if 'popup' in msg and call_or_available:
        return 'popup'
    
    
    if 'ebay' in msg and call_or_available:
        return 'ebay'
    
    
    if 'delta' in msg and call_or_available:
        return 'delta'
    
    
    if 'bsod' in msg and call_or_available:
        return 'bsod'
    
    if 'refund' in msg and call_or_available:
        return 'refund'
    
    
    if ' cc details ' in msg or 'indian cc' in msg or ' cc data ' in msg:
        return 'query for call center'
    
    
    if 'website traffic available' in msg or (call and available):
        return 'calls available'
    
    return False

df['website_developer'] = df['message_text'].apply(website_developer)
temp = df[df['website_developer'] != False]
print(temp.shape)


print(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)
print(temp['total_comment'].sum(axis=0, skipna=True))

(5167, 13)
(1411, 12)
(525, 12)
1908


In [6]:
def call_center(msg):
    looking = 'looking' in msg or 'hiring' in msg or 'required' in msg or 'need' in msg
    agent = 'agent' in msg or 'fresher' in msg
    
    if agent and looking:
        return 'need agents'
    
    return False

df['call_center'] = df['message_text'].apply(call_center)
temp = df[df['call_center'] != False]
print(temp.shape)

print(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)
print(temp['total_comment'].sum(axis=0, skipna=True))

(1322, 14)
(516, 13)
(245, 13)
1005


In [7]:
def victim_data(msg):
    lead = 'lead' in msg or 'data' in msg
    email = 'email' in msg and lead
    refund = 'refund' in msg and lead
    ivr = 'ivr' in msg and lead
    fresh = 'fresh' in msg and lead
    data = 'data available' in msg
    
    smtp = 'smtp' in msg
    bulk_sms = 'bulk sms' in msg
    blasting = 'email' in msg and 'blast' in msg

    if email or refund or ivr or fresh or data:
        return 'leads'
    
    if blasting or smtp or bulk_sms:
        return 'blasting'
    return False

df['victim_data'] = df['message_text'].apply(victim_data)
temp = df[df['victim_data'] != False]

print(temp.shape)

print(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)
print(temp['total_comment'].sum(axis=0, skipna=True))

(2977, 15)
(981, 14)
(367, 14)
1521


In [8]:
def toll_free_number(msg):
    available = 'available' in msg or 'avail' in msg or 'providing' in msg or 'get unlimited call' in msg
    toll_free_number = ('tfn' in msg or 'toll free' in msg or 'toll-free' in msg) and available
    did = 'did' in msg and available
    
    if toll_free_number or did:
        return True
    return False

df['toll_free_number'] = df['message_text'].apply(toll_free_number)
temp = df[df['toll_free_number'] != False]

print(temp.shape)

print(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)
print(temp['total_comment'].sum(axis=0, skipna=True))

(1324, 16)
(295, 15)
(125, 15)
246


In [9]:
def seo(msg):
    if ' seo' in msg or 'social media promotion' in msg or 'social media service' in msg or 'digital maketing' in msg:
        return 'social media promotion'
    return False
    
df['social_media_promotion'] = df['message_text'].apply(seo)
temp = df[df['social_media_promotion'] != False]

print(temp.shape)

print(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)
print(temp['total_comment'].sum(axis=0, skipna=True))

(331, 17)
(108, 16)
(70, 16)
74


In [10]:
def other(msg):
    logic = empty_post(msg) or money_laundering(msg) or website_developer(msg) or call_center(msg) or victim_data(msg) or seo(msg) or toll_free_number(msg)
        
    if not logic:
        return True
    return False
    

df['other'] = df['message_text'].apply(other)
temp = df[df['other'] == True]

print(temp.shape)
print(pd.pivot_table(temp, index=['message_text'], aggfunc=pd.Series.count).shape)
print(pd.pivot_table(temp, index=['actor_id'], aggfunc=pd.Series.count).shape)
print(temp['total_comment'].sum(axis=0, skipna=True))

(7356, 18)
(3578, 17)
(1498, 17)
7292


In [11]:
df.to_csv('labelled_posts.tsv', sep='\t', header=True, index=False)